## Разметка и извлечение именованных сущностей

Тема «POS-tagger и NER»
Задание
 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы, сделать выводы  


Задание 2. Проверить, насколько хорошо работает NER
Данные брать из Index of /pub/named_entities
проверить NER из nltk/spacy/deeppavlov.
написать свой NER, попробовать разные подходы.
передаём в сетку токен и его соседей.
передаём в сетку только токен.
свой вариант.
сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.

### 3.1. Разметка слов с помощью частей речи (Parts-Of-Speech)

In [38]:
#nltk.download() 
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline

In [39]:
import warnings
warnings.filterwarnings("ignore")

In [40]:
nltk.help.upenn_tagset('RB')
nltk.help.upenn_tagset('NN')
nltk.help.upenn_tagset('VB')

RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
VB: verb, base form
    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...


#### Сравнение POS тэггеров

POS-тэггер обрабатывает последовательность слов и определяет тэг части речи для каждого слова. Сравним работу нескольких тэггеров библиотеки nltk.tag. Проверять работоспособность теггеров будем на корпусе nltk.corpus.brown.

Отобразим распределение тэгов в корпусе brown. Можем видеть, что тэг "NN" наиболее популярный.

Корпус будет поделен на train и test, т.к. некоторым тэггерам необходимо обучение. Разметку визуально будем оценивать на примере test_sent, тестового предложения.

In [41]:
# %pip install pyconll
# %pip install corus
# %pip install -U spacy
# %pip install slovnet
# %pip install deeppavlov
# %pip install ipymarkup
# %pip install sklearn_crfsuite

In [42]:
# %%python3 -m spacy download ru_core_news_sm
# %%python3 -m deeppavlov install squad_bert
# %%python3 -m deeppavlov install ner_ontonotes

In [43]:
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import pyconll
import nltk
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
import os
import corus
import deeppavlov
from deeppavlov import configs, build_model
import spacy
from spacy import displacy
import ru_core_news_sm
import pandas as pd
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from razdel import tokenize
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score

[nltk_data] Downloading package punkt to /home/ekaterina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/ekaterina/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/ekaterina/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ekaterina/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [45]:
!wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

/bin/bash: /home/ekaterina/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
--2023-12-17 21:29:50--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 40736599 (39M) [text/plain]
Сохранение в: «ru_syntagrus-ud-train.conllu»

ru_syntagrus-ud-tra 100%[===================>]  38,85M  1,90MB/s    за 28s     

2023-12-17 21:30:21 (1,40 MB/s) - «ru_syntagrus-ud-train.conllu» сохранён [40736599/40736599]

/bin/bash: /home/ekaterina/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
--2023-12-17 21:30:22--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian

In [53]:
# Загружаем данные
data_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
data_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [54]:
fdata_train = []
for sent in data_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in data_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in data_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [55]:
fdata_train[:10]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')],
 [('В', 'ADP'),
  ('приемной', 'NOUN'),
  ('его', 'PRON'),
  ('с', 'ADP'),
  ('утра', 'NOUN'),
  ('ожидали', 'VERB'),
  ('посетители', 'NOUN'),
  (',', 'PUNCT'),
  ('-', 'PUNCT'),
  ('кое-кто', 'PRON'),
  ('с', 'ADP'),
  ('важными', 'ADJ'),

In [56]:
default_tagger = nltk.DefaultTagger('NN')
default_acc = default_tagger.evaluate(fdata_test)

unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate(fdata_test)

print(f'Accuracy:\nDefault Tagger: {round(default_acc, 3)},\nUnigram Tagger: {round(unigram_acc, 3)},\nBigram Tagger: {round(bigram_acc, 5)},\n'
      f'Trigram Tagger: {round(trigram_acc, 3)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 5)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 5)},\n')

Accuracy:
Default Tagger: 0.0,
Unigram Tagger: 0.824,
Bigram Tagger: 0.60939,
Trigram Tagger: 0.178,
Bigram and Unigram Tagger: 0.82928,
Trigram, Bigram and Unigram Tagger: 0.82914,



In [57]:
def union_taggers(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = union_taggers(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test)

0.827905462595221

## Создание своего тэггера

In [58]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(' ' if tok[0] is None else tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [61]:
train_tok[:10], train_label[:10]

(['Анкета',
  '.',
  'Начальник',
  'областного',
  'управления',
  'связи',
  'Семен',
  'Еремеевич',
  'был',
  'человек'],
 ['NOUN',
  'PUNCT',
  'NOUN',
  'ADJ',
  'NOUN',
  'NOUN',
  'PROPN',
  'PROPN',
  'AUX',
  'NOUN'])

In [77]:
le = LabelEncoder()
y_train = le.fit_transform(train_label)
y_train

array([ 7, 13,  7, ...,  1, 11, 13])

In [78]:
y_test = le.transform(test_label)
y_test

array([ 7, 13,  1, ...,  0,  7, 13])

In [79]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [81]:
count_labels = len(set(y_train))
count_labels

18

In [82]:
%time

vectorizers = [CountVectorizer(ngram_range=(1, 5), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 5), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 5), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 5), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [92]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.utils import to_categorical

In [84]:
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy

In [93]:
# Build the model.
def create_model(optimizer='adam'):
    model = Sequential([
      Dense(512, input_shape=(5000,), activation='relu'),
      Dense(256, activation='relu'),
      Dense(18, activation='sigmoid')
    ])

    # Compile the model.
    model.compile(
      optimizer=optimizer,
      loss='categorical_crossentropy',
      metrics=['accuracy'],
    )
    return model


model = create_model()

Epoch 1/5
13319/13319 [==============================] - 266s 20ms/step - loss: 0.8992 - accuracy: 0.6875
Epoch 2/5
13319/13319 [==============================] - 265s 20ms/step - loss: 0.8003 - accuracy: 0.7158
Epoch 3/5
13319/13319 [==============================] - 266s 20ms/step - loss: 0.7831 - accuracy: 0.7204
Epoch 4/5
13319/13319 [==============================] - 262s 20ms/step - loss: 0.7761 - accuracy: 0.7222
Epoch 5/5
4800/4800 [==============================] - 4s 716us/step - loss: 0.9979 - accuracy: 0.6764
*****************************
Loss - 0.9979, Metrics - 0.6764


In [94]:
# Train the model.
model.fit(X_train, to_categorical(y_train), epochs=15, batch_size=64)

# Evaluate the model.
loss,metrics = model.evaluate(X_test, to_categorical(y_test))
print('*****************************')
print(f'Loss - {np.round(loss,4)}, Metrics - {np.round(metrics,4)}')

Epoch 1/15
6660/6660 [==============================] - 135s 20ms/step - loss: 0.7638 - accuracy: 0.7247
Epoch 2/15
6660/6660 [==============================] - 135s 20ms/step - loss: 0.7617 - accuracy: 0.7249
Epoch 3/15
6660/6660 [==============================] - 135s 20ms/step - loss: 0.7600 - accuracy: 0.7250
Epoch 4/15
6660/6660 [==============================] - 135s 20ms/step - loss: 0.7587 - accuracy: 0.7253
Epoch 5/15
6660/6660 [==============================] - 135s 20ms/step - loss: 0.7581 - accuracy: 0.7255
Epoch 6/15
6660/6660 [==============================] - 135s 20ms/step - loss: 0.7576 - accuracy: 0.7256
Epoch 7/15
6660/6660 [==============================] - 135s 20ms/step - loss: 0.7567 - accuracy: 0.7259
Epoch 8/15
6660/6660 [==============================] - 135s 20ms/step - loss: 0.7566 - accuracy: 0.7257
Epoch 9/15
6660/6660 [==============================] - 136s 20ms/step - loss: 0.7560 - accuracy: 0.7259
Epoch 10/15
6660/6660 [==============================] 

Вывод: наш тэггер обучается, но медлено, необходимо увеличение количества эпох обучения или перенастройка параметров нейросети

## NER

In [104]:
!python -m spacy download ru_core_news_lg

/bin/bash: /home/ekaterina/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     |████████████████████████████████| 513.4 MB 57 kB/s  eta 0:00:01     |█████████████████▎              | 277.8 MB 2.0 MB/s eta 0:02:00     |██████████████████████████▍     | 423.3 MB 1.9 MB/s eta 0:00:47     |███████████████████████████▉    | 446.2 MB 1.6 MB/s eta 0:00:43
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [105]:
!pip install corus razdel navec slovnet deeppavlov

/bin/bash: /home/ekaterina/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [106]:
import nltk
import os
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

import spacy
from spacy import displacy
import en_core_web_md
import ru_core_news_lg

import corus
from corus import load_ne5
from razdel import tokenize

import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /home/ekaterina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ekaterina/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/ekaterina/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/ekaterina/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [107]:
dataset_for_nltk = []

for filename in os.scandir("Collection5/"):
    if filename.is_file():
        name, ext = os.path.splitext(filename.path)
        if ext == ".txt":
          with(open(filename.path,"r",encoding='utf-8') as f):
            text = f.read()
            tokens = text.split()
          
          tagged_tokens = {}

          with(open(name + ".ann","r",encoding='utf-8') as f):
            for line in f:
              parts = line.split()
              tag = parts[1]
              name = ' '.join(parts[4:])
              tagged_tokens[name] = tag

          dataset_for_nltk.append((text, tagged_tokens))


In [108]:
for entry in dataset_for_nltk:
  pred = [(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(entry[0]))) if hasattr(chunk, 'label')]
  print(entry[1])
  print(pred)
  break

{'Сванидзе': 'PER', 'Сноудена': 'PER', 'ОП': 'ORG', 'Николай Сванидзе': 'PER', 'Эдварда Сноудена': 'PER', 'Общественной палаты (ОП)': 'ORG', 'РФ': 'GEOPOLIT', 'Общественной палате': 'ORG', 'Госдумы': 'ORG', 'Александр Сидякин': 'PER', 'Booz Allen Hamilton': 'ORG', 'Центрального разведывательного управления': 'ORG', 'США': 'GEOPOLIT', 'Эдвард Сноуден': 'PER', 'СМИ': 'MEDIA', 'Общественной палаты': 'ORG', 'ФСБ': 'ORG', 'России': 'GEOPOLIT'}
[('Сванидзе', 'PERSON'), ('Николай Сванидзе', 'PERSON'), ('Эдварда Сноудена', 'PERSON'), ('Николай Сванидзе', 'PERSON'), ('Госдумы Александр Сидякин', 'PERSON'), ('Hamilton', 'PERSON'), ('Эдвард Сноуден', 'PERSON'), ('Нобелевку', 'PERSON')]


In [111]:
nlp = ru_core_news_lg.load()
doc = nlp(dataset_for_nltk[1][0])
displacy.render(doc, jupyter=True, style='ent')

In [112]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [113]:
!locale

/bin/bash: /home/ekaterina/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
LANG=ru_RU.UTF-8
LANGUAGE=
LC_CTYPE="ru_RU.UTF-8"
LC_NUMERIC="ru_RU.UTF-8"
LC_TIME="ru_RU.UTF-8"
LC_COLLATE="ru_RU.UTF-8"
LC_MONETARY="ru_RU.UTF-8"
LC_MESSAGES="ru_RU.UTF-8"
LC_PAPER="ru_RU.UTF-8"
LC_NAME="ru_RU.UTF-8"
LC_ADDRESS="ru_RU.UTF-8"
LC_TELEPHONE="ru_RU.UTF-8"
LC_MEASUREMENT="ru_RU.UTF-8"
LC_IDENTIFICATION="ru_RU.UTF-8"
LC_ALL=


In [117]:
def load_text_patched(path):
    # do not convert \r\n to \n
    with open(path, newline='', encoding='utf-8') as file:
        return file.read()

dir = 'Collection5/'
# load_ne5 do not accept encoding, but Colab sometimes fails to default to utf-8 for unknown reasons
corus.ne5.load_text = load_text_patched
records = load_ne5(dir)

In [118]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [119]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [127]:
df_words.head()

,word,tag
0,Сванидзе,PER
1,не,OUT
2,одобрил,OUT
3,идею,OUT
4,о,OUT


In [120]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [121]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [122]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [123]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 128

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    # ngrams=(1, 5),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [124]:
embedding_dim = 256

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(512, activation='relu')
        self.fc2 = Dense(256, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [125]:
model = modelNER()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['acc'])

In [126]:
model.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 748s 60ms/step - loss: 0.2782 - acc: 0.9196 - val_loss: 0.2169 - val_acc: 0.9356
Epoch 2/5
12444/12444 [==============================] - 722s 58ms/step - loss: 0.1182 - acc: 0.9644 - val_loss: 0.2586 - val_acc: 0.9384
Epoch 3/5
12444/12444 [==============================] - 723s 58ms/step - loss: 0.1048 - acc: 0.9668 - val_loss: 0.2524 - val_acc: 0.9354
Epoch 4/5
12444/12444 [==============================] - 724s 58ms/step - loss: 0.1002 - acc: 0.9676 - val_loss: 0.2919 - val_acc: 0.8766
Epoch 5/5
12444/12444 [==============================] - 742s 60ms/step - loss: 0.0980 - acc: 0.9680 - val_loss: 0.2916 - val_acc: 0.9342


In [128]:
y_pred1 = model.predict(valid_x)
y_pred = np.argmax(y_pred1, axis=1)

print(f'Precision score {precision_score(valid_y, y_pred , average="macro")}')
print(f'Recall score {recall_score(valid_y, y_pred , average="macro")}')
print(f'F1 score {f1_score(valid_y, y_pred , average="macro")}')

2074/2074 [==============================] - 3s 1ms/step
Precision score 0.915441425182583
Recall score 0.7568160395613867
F1 score 0.8183895331000081


Вывод: модель работает неплохо, f1 составляет 0,82. Можно улучшить модель, изменив параметры для повышения recall.